<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 5px; height: 50px"> 

#   Personalizing Music Video Recommendations with Emotional Intelligence

> Capstone Project: Lionel Foo

---

#### <b> Notebook: 04 Building an Emotion-based Music Video Recommendation System </b>

<b> The objective is to design an emotion-centric music video recommendation system. </b>

Music holds a profound capacity to elicit human emotions. [1] The field of music recommendation is particularly influenced by this emotional aspect, as individuals often gravitate towards music that mirrors their current emotional state. [2] [3]

The proposed system will analyze YouTube users’ comments to discern their emotional state and subsequently recommend Youtube music videos. These recommendations aim to either resonate with the users’ current emotional state or aid them in navigating through negative emotions.

The system will prioritize user well-being by suggesting songs that harmonize with their current mood. It will evaluate users’ emotional needs based on their comments and recommend the most “beneficial” music videos instead of the most “matched” ones. A beneficial music video is defined by two criteria:
* It aligns with the user’s emotional state, thereby fulfilling their emotional needs.
* If the user’s overall mood is negative, the recommended music video should have the potential to uplift the user’s mood. This implies that the music should be slightly more positive than the user’s current mood. However, to prevent causing emotional discomfort or resistance to the recommendation, the degree of positivity should be delicately balanced. [4]

This approach ensures that the system is not just recommending music videos, but also contributing to the users’ emotional well-being.

<font size="1.5">

References: 

[1] University of Turku. Music can evoke strong emotions - the same melody can lead to shivers down the spine or feelings of joy. <br>https://www.utu.fi/en/news/press-release/music-can-evoke-strong-emotions-the-same-melody-can-lead-to-shivers-down-the-spine-or-feelings-of-joy

[2] IEEE Xplore. Mood-Based Music Recommendation System Using Supervised Learning. <br>https://ieeexplore.ieee.org/document/9256342

[3] Springer Link. Induced Emotion-Based Music Recommendation through Reinforcement Learning. <br>https://link.springer.com/chapter/10.1007/978-3-030-35288-2_20

[4] Science Direct. An Emotion-based Personalized Music Recommendation Framework for Emotion Improvement. <br>https://www.sciencedirect.com/science/article/abs/pii/S0306457322003570


</font>


---

<b> 1. Import Libraries

In [147]:
# Imports: standard
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import pickle
from scipy.spatial import KDTree
from sklearn.metrics.pairwise import cosine_similarity


---

<b> 2. Predicting emotions of each Scraped Youtube Comment </b>
* In notebook 2C we build an emotion-centric text-based classifier (RNN with GloVe Word Embeddings) that can classify text into five classes: Sadness, Joy, Love, Anger, and Fear.

<b> Step Outline </b>

1. **Load the Model and Tokenizer:** load the pre-trained emotion classifier model and the tokenizer used during the model training.
2. **Import YouTube Comments:** import a dataset of scraped and cleaned YouTube comments. This dataset contains approx 50,000 comments, with 1,000 comments each from a list of 100 music videos.
3. **Prepare Text for Model:** create a new column ‘text_to_model’ that replicates the ‘text’ column. This column will be used to prepare the text data for the model. Tokenize the text and pad the sequences to ensure they all have the same length.
4. **Predict Emotions:** use the emotion classifier model to predict the emotions expressed in the comments. The model outputs both the predicted labels and the probabilities for each emotion class.
5. **Map Labels to Emotions:** map the predicted labels to their corresponding emotions using a predefined dictionary.
6. **Clean Up DataFrame:** drop the ‘text_to_model’ column from the DataFrame as it’s no longer needed.
7. **Final Output:** The final output is a DataFrame that includes the original YouTube comments along with the predicted emotion and the probabilities for each emotion class.


In [148]:
# Load the model
emotion_classifier_model = load_model('Model/model_emotions.keras')

# Load the tokenizer
with open('Model/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [149]:
# Import cleaned youtube comments
youtube_comments = pd.read_csv("Data/youtube_comments_clean.csv")
youtube_comments.head(3)

,author,updated_at,like_count,text,video_id,public
0,@Royalchess1,2024-02-24T09:02:54Z,0,ms miley i have alot of mixed emotions while w...,G7KNmW9a75Y,True
1,@mikerooney7600,2024-02-24T09:01:41Z,0,i love this song it makes every day better,G7KNmW9a75Y,True
2,@Ganesh-zs1iv,2024-02-24T08:07:13Z,0,it is feb and still i am watching it,G7KNmW9a75Y,True


In [150]:
# Check there are no null rows under 'text' columns
youtube_comments['text'].isnull().sum()

0

In [151]:
# Dropping unnecessary columns, renaming 'video_id', and organising relevant columns
youtube_comments.drop(columns=['updated_at', 'like_count', 'public'], inplace=True)
youtube_comments.rename(columns={'video_id': 'comment_origin_video_id'}, inplace=True)
youtube_comments = youtube_comments[['author', 'comment_origin_video_id', 'text']]
youtube_comments.head(3)

,author,comment_origin_video_id,text
0,@Royalchess1,G7KNmW9a75Y,ms miley i have alot of mixed emotions while w...
1,@mikerooney7600,G7KNmW9a75Y,i love this song it makes every day better
2,@Ganesh-zs1iv,G7KNmW9a75Y,it is feb and still i am watching it


In [152]:
# Create a new column 'text_to_model' that replicates 'text'
youtube_comments['text_to_model'] = youtube_comments['text']

In [153]:
# Tokenize and pad sequences
youtube_comments['text_to_model'] = youtube_comments['text_to_model'].apply(lambda x: tokenizer.texts_to_sequences([x]))
youtube_comments['text_to_model'] = youtube_comments['text_to_model'].apply(lambda x: pad_sequences(x, maxlen=50))

In [154]:
# Predict labels and probabilities
predictions = emotion_classifier_model.predict(np.concatenate(youtube_comments['text_to_model'].values))
youtube_comments['label'] = predictions.argmax(axis=1)
youtube_comments['probability_sadness'] = predictions[:, 0]
youtube_comments['probability_joy'] = predictions[:, 1]
youtube_comments['probability_love'] = predictions[:, 2]
youtube_comments['probability_anger'] = predictions[:, 3]
youtube_comments['probability_fear'] = predictions[:, 4]

1745/1745 [==============================] - 6s 3ms/step


In [155]:
# Map labels to emotions
emotion_dict = {0: 'Sadness', 1: 'Joy', 2: 'Love', 3: 'Anger', 4: 'Fear'}
youtube_comments['emotion'] = youtube_comments['label'].map(emotion_dict)

In [156]:
youtube_comments.tail(3)

,author,comment_origin_video_id,text,text_to_model,label,probability_sadness,probability_joy,probability_love,probability_anger,probability_fear,emotion
55826,@69jdwalt,vhumOLNSSJY,no one should be taken over after this is a ho...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",3,0.001944,0.040106,0.000712,0.953556,0.003682,Anger
55827,@ZShorts-gf7tz,vhumOLNSSJY,absolutely amazing song nf is not just talente...,"[[898, 33, 482, 605, 22, 28, 44, 210, 63, 98, ...",1,0.000020,0.996426,0.003328,0.000060,0.000166,Joy
55828,@TurdFPh.D,vhumOLNSSJY,people rely on you people you have never met y...,"[[13, 108, 279, 38, 1421, 3, 100, 28, 4, 1102,...",4,0.059979,0.044040,0.033233,0.236552,0.626196,Fear


In [157]:
# Drop the 'text_to_model' column from the DataFrame:
youtube_comments = youtube_comments.drop(columns = 'text_to_model')
# Reset index:
youtube_comments = youtube_comments.reset_index(drop=True)

In [158]:
# export data frame with emotions classified youtube comments as csv:
youtube_comments.to_csv('Data/youtube_comments_emotion_predictions.csv', index=False)


---

<b> 3. analyze the emotional content of music videos based on their lyrics </b>

<b> Step Outline </b>

1. **Import Music Video Transcripts:** import a dataset of cleaned and chunked transcripts (in Notebook 3A) from 100 popular music videos from the 2020s.
2. **Prepare Text for Model:** create a new column ‘text_to_model’ that replicates the ‘transcript’ column. This column will be used to prepare the text data for the model. tokenize the text and pad the sequences to ensure they all have the same length.
3. **Predict Emotions:** use the emotion classifier model to predict the emotions expressed in the transcripts. The model outputs both the predicted labels and the probabilities for each emotion class.
4. **Map Labels to Emotions:** map the predicted labels to their corresponding emotions using predefined dictionary above.
5. **Combining back the chunks**
- (a) **Concatenate Transcripts:** concatenate the transcripts for each video (same video_id) into a single string.
- (b) **Average Probabilities:** calculate the average probability for each emotion class for each video.
- (c) **Update Labels:** update the label for each video to the emotion class with the highest average probability.
- (d) **Map Labels to Emotions:** map the updated labels to their corresponding emotions using the predefined dictionary.
6. **Clean Up Dataframe:** drop duplicate rows from the DataFrame and reset the index.
7. **Final Output** The final output is a DataFrame that includes the transcripts of the music videos along with the predicted emotion and the average probabilities for each emotion class.

In [159]:
# Import cleaned youtube comments
youtube_mv_transcripts = pd.read_csv("Data/youtube_mv_transcript_clean.csv")
youtube_mv_transcripts.head(3)

,video_id,transcript
0,G7KNmW9a75Y,we were good we were gold kind of dream that...
1,G7KNmW9a75Y,started to cry but then remembered i i can bu...
2,G7KNmW9a75Y,yeah i can love me better than you can can lo...


In [160]:
# Create a new column 'text_to_model' that replicates 'text'
youtube_mv_transcripts['text_to_model'] = youtube_mv_transcripts['transcript']
youtube_mv_transcripts.head(3)

,video_id,transcript,text_to_model
0,G7KNmW9a75Y,we were good we were gold kind of dream that...,we were good we were gold kind of dream that...
1,G7KNmW9a75Y,started to cry but then remembered i i can bu...,started to cry but then remembered i i can bu...
2,G7KNmW9a75Y,yeah i can love me better than you can can lo...,yeah i can love me better than you can can lo...


In [161]:
# Tokenize and pad sequences
youtube_mv_transcripts['text_to_model'] = youtube_mv_transcripts['text_to_model'].apply(lambda x: tokenizer.texts_to_sequences([x]))
youtube_mv_transcripts['text_to_model'] = youtube_mv_transcripts['text_to_model'].apply(lambda x: pad_sequences(x, maxlen=50))

In [162]:
# Predict labels and probabilities
predictions_mv = emotion_classifier_model.predict(np.concatenate(youtube_mv_transcripts['text_to_model'].values))
youtube_mv_transcripts['label'] = predictions_mv.argmax(axis=1)
youtube_mv_transcripts['probability_sadness'] = predictions_mv[:, 0]
youtube_mv_transcripts['probability_joy'] = predictions_mv[:, 1]
youtube_mv_transcripts['probability_love'] = predictions_mv[:, 2]
youtube_mv_transcripts['probability_anger'] = predictions_mv[:, 3]
youtube_mv_transcripts['probability_fear'] = predictions_mv[:, 4]

31/31 [==============================] - 0s 4ms/step


In [163]:
# Map labels to emotions
youtube_mv_transcripts['emotion'] = youtube_mv_transcripts['label'].map(emotion_dict)

In [164]:
# Check output
youtube_mv_transcripts

,video_id,transcript,text_to_model,label,probability_sadness,probability_joy,probability_love,probability_anger,probability_fear,emotion
0,G7KNmW9a75Y,we were good we were gold kind of dream that...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 77, 120, 117, ...",3,3.811742e-02,0.049064,0.000617,9.033179e-01,8.883808e-03,Anger
1,G7KNmW9a75Y,started to cry but then remembered i i can bu...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 177, 4, 639, 22, ...",1,2.435994e-03,0.955003,0.011286,1.009263e-02,2.118289e-02,Joy
2,G7KNmW9a75Y,yeah i can love me better than you can can lo...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1420, 1, 44, 8...",0,7.336176e-01,0.258982,0.000836,6.150512e-03,4.139581e-04,Sadness
3,G7KNmW9a75Y,no remorse no regret i forgive every word yo...,"[[0, 0, 0, 0, 0, 0, 0, 93, 3268, 93, 1325, 1, ...",1,5.313131e-03,0.939050,0.008151,1.419929e-02,3.328642e-02,Joy
4,G7KNmW9a75Y,talk to myself for hours say things you do no...,"[[0, 0, 0, 0, 0, 0, 0, 0, 272, 4, 51, 18, 475,...",1,2.517612e-01,0.404417,0.071435,1.885957e-01,8.379038e-02,Joy
...,...,...,...,...,...,...,...,...,...,...
971,vhumOLNSSJY,from how i feel but i am too proud to open up...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 54, 1, 2, ...",1,1.794297e-07,0.999994,0.000006,1.726046e-07,1.692862e-07,Joy
972,vhumOLNSSJY,who i would be if i was happy do not what is ...,"[[0, 0, 0, 0, 0, 0, 0, 0, 70, 1, 52, 27, 50, 1...",1,1.492461e-01,0.752897,0.023806,4.882041e-02,2.523041e-02,Joy
973,vhumOLNSSJY,like i do not care what anyone else thinks w...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,4.196371e-07,0.999992,0.000007,1.545425e-07,8.379989e-07,Joy
974,vhumOLNSSJY,to pick me up and pull me out this hole i am t...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1001, 20, 4...",1,2.898759e-02,0.956532,0.006740,3.617591e-03,4.122662e-03,Joy


In [165]:
# Drop the 'text_to_model' column from the DataFrame:
youtube_mv_transcripts = youtube_mv_transcripts.drop(columns = 'text_to_model')

In [166]:
# Concatenate the transcripts and text_to_model
youtube_mv_transcripts['transcript'] = youtube_mv_transcripts.groupby('video_id')['transcript'].transform(lambda x : ' '.join(x))

# Average the probabilities
youtube_mv_transcripts['probability_sadness'] = youtube_mv_transcripts.groupby('video_id')['probability_sadness'].transform('mean')
youtube_mv_transcripts['probability_joy'] = youtube_mv_transcripts.groupby('video_id')['probability_joy'].transform('mean')
youtube_mv_transcripts['probability_love'] = youtube_mv_transcripts.groupby('video_id')['probability_love'].transform('mean')
youtube_mv_transcripts['probability_anger'] = youtube_mv_transcripts.groupby('video_id')['probability_anger'].transform('mean')
youtube_mv_transcripts['probability_fear'] = youtube_mv_transcripts.groupby('video_id')['probability_fear'].transform('mean')

# Update the label
youtube_mv_transcripts['label'] = youtube_mv_transcripts[['probability_sadness', 'probability_joy', 'probability_love', 'probability_anger', 'probability_fear']].idxmax(axis=1)

# Create a dictionary to map probability column names to labels
prob_to_label = {'probability_sadness': 0, 'probability_joy': 1, 'probability_love': 2, 'probability_anger': 3, 'probability_fear': 4}

# Map the string labels to their integer representations
youtube_mv_transcripts['label'] = youtube_mv_transcripts['label'].map(prob_to_label)

#Update the emotions
youtube_mv_transcripts['emotion'] = youtube_mv_transcripts['label'].map(emotion_dict)

# Drop duplicates
youtube_mv_transcripts.drop_duplicates(subset='video_id', keep='first', inplace=True)

In [167]:
# Reset index:
youtube_mv_transcripts = youtube_mv_transcripts.reset_index(drop=True)

In [168]:
# Check final dataframe for youtube mv transcripts
youtube_mv_transcripts

,video_id,transcript,label,probability_sadness,probability_joy,probability_love,probability_anger,probability_fear,emotion
0,G7KNmW9a75Y,we were good we were gold kind of dream that...,1,0.185217,0.605823,0.024522,0.165062,0.019376,Joy
1,Oa_RSwwpPaA,for a while that it was rough but lately i hav...,1,0.330111,0.335034,0.009799,0.069333,0.255724,Joy
2,8UMnAIaBofo,tell death to us part tell death to us part i ...,3,0.299442,0.142625,0.108606,0.356857,0.092470,Anger
3,Iq8h3GEe22o,i do not like no whips and ch can not me down ...,3,0.412854,0.015061,0.014315,0.549594,0.008176,Anger
4,H5v3kku4y6Q,holding me back gravity holding me back i wa...,0,0.340206,0.305491,0.046790,0.092395,0.215117,Sadness
...,...,...,...,...,...,...,...,...,...
95,uS_y_65CcpA,you want the girl with the small waist and th...,0,0.471563,0.204420,0.008680,0.190082,0.125255,Sadness
96,QBPE2fZsVYU,i have been dreaming about the west coast got ...,1,0.281542,0.412919,0.082388,0.133559,0.089593,Joy
97,JKrDdsgXuso,as you promised me that i was more than all th...,1,0.325391,0.389652,0.040207,0.210264,0.034487,Joy
98,X0lRcz_IuPQ,i believe in ghosts most people do not i swe...,4,0.226523,0.335049,0.005566,0.080700,0.352162,Fear


In [169]:
# Plot a pir chart to view the Emotion distribution of the 100 Music Videos
# Create a copy of your DataFrame
df = youtube_mv_transcripts.copy()

# Count the occurrences of each emotion
emotions_distribution = df['emotion'].value_counts().reset_index()
emotions_distribution.columns = ['Emotion', 'Count']

# Sort the DataFrame to make 'Joy' and 'Love' adjacent
emotions_distribution = emotions_distribution.sort_values(by='Emotion', key=lambda col: col != 'Love')

# Plot the pie chart
fig = px.pie(emotions_distribution, names='Emotion', values='Count', title='Emotion distribution of Music Videos')
fig.update_traces(textposition='inside', textinfo='label+value')

# Update the layout with a specific width (in pixels)
fig.update_layout(width=500)  # Adjust this value to make the figure narrower or wider
fig.show()

In [170]:
# export data frame with emotions classified mv transcript as csv:
youtube_mv_transcripts.to_csv('Data/youtube_mv_predicted_emotion.csv', index=False)


---

<b> 4. Developing a rule-based recommender system that matches the emotions of each user comment to a list of 5 music videos. </b>
* The rules for recommending the music videos differ based on whether the user is deemed to have positive or negative emotions. Here’s a summary of the steps:

<b> Step Outline </b>

1. **Define Emotions** categorize emotions into two groups: positive (Joy, Love) and negative (Sadness, Anger, Fear).
2. **Get Original Video ID:** identify the video from which the user’s comment originated.
3. **Filter Out Original Video:** exclude the original video from the list of potential recommendations.
4. **Recommendation Rules** set up different rules for recommending music videos based on the user’s emotion.
- **Positive Emotion**: If the user’s emotion is positive, look for music videos that evoke a similar emotion. For example, if a user’s comment is classified as ‘Joy’ with a certain probability, recommend 5 music videos whose ‘Joy’ probabilities are closest to that of the user’s comment. This is done using a method called KDTree, which is a way of finding the closest points (in this case, emotion probabilities) in a space.
- **Negative Emotion**: If the user’s emotion is negative, look for music videos that can help uplift their mood. Form a ‘vector’ or a set of values for the comment, consisting of the probabilities of the three negative emotions. Do the same for all 100 music videos. Then, calculate the cosine similarity between the comment vector and each music video vector. Cosine similarity is a measure of how similar two vectors are, and in this case, it’s used to find music videos whose emotion probabilities are most similar to the user’s comment. However, the system will only consider those videos whose sum of negative emotion probabilities is at least 0.1 less than that of the user’s comment. This ensures that the recommended videos are slightly more positive than the user’s current mood. The 5 music videos with the smallest cosine similarity scores are recommended.
5. **Print User Comment, Emotion, and Emotion Probability:** display the user’s comment, the detected emotion, and the probability of that emotion. For negative emotions, instead of probability of that emotion, display the user's comment sum of negative emotion probabilities
6. **Print Recommended Video IDs and Their Emotion Probabilities:** display the IDs of the recommended videos and their corresponding emotion probabilities. For negative emotions, instead display the sum of the negative emotion probabilities and the cosine similarity scores.
7. **Convert Video IDs to YouTube URLs:** convert the IDs of the recommended videos into YouTube URLs.
8. **Return Recommended Videos:** return the recommended videos.

In [171]:
# Define a function that Recommends music videos based on user comment and emotion, utilizing transcript and comment data
def recommend_music_videos(user_comment, user_emotion, youtube_mv_transcripts, youtube_comments):
    # Define the emotions
    positive_emotions = ['Joy', 'Love']
    negative_emotions = ['Sadness', 'Anger', 'Fear']
    negative_probs = ['probability_' + emo.lower() for emo in negative_emotions]

    # Get the video id of the original video the comment was scraped from
    origin_video_id = user_comment['comment_origin_video_id']

    # Filter out the original video from the transcripts dataframe
    filtered_transcripts = youtube_mv_transcripts[youtube_mv_transcripts['video_id'] != origin_video_id]

    recommended_videos = None
    if user_emotion in positive_emotions:
        # For positive emotions, find the 5 closest emotion probabilities
        tree = KDTree(filtered_transcripts[['probability_' + user_emotion.lower()]])
        dist, idx = tree.query([[user_comment['probability_' + user_emotion.lower()]]], k=min(5, len(filtered_transcripts)))
        recommended_videos = filtered_transcripts.iloc[idx[0]]

        # Print the user comment, emotion, and emotion probability
        print(f"User comment: {user_comment['text']}")
        print(f"User emotion: {user_emotion}")
        print(f"User emotion probability: {user_comment['probability_' + user_emotion.lower()]}")

    elif user_emotion in negative_emotions:
        # For negative emotions, calculate cosine similarity
        user_vector = user_comment[negative_probs].values.reshape(1, -1)
        
        # Filter out videos with sum of negative emotion probabilities that are not at least 0.1 than the user's comment
        filtered_transcripts = filtered_transcripts[
            (filtered_transcripts[negative_probs].sum(axis=1) < user_comment[negative_probs].sum() - 0.1)
        ]

        # Calculate cosine similarity
        filtered_transcripts['cosine_similarity'] = filtered_transcripts.apply(lambda row: cosine_similarity(user_vector, row[negative_probs].values.reshape(1, -1))[0][0], axis=1)

        # Find the 5 closest emotion probabilities
        if len(filtered_transcripts) > 0:
            recommended_videos = filtered_transcripts.nsmallest(5, 'cosine_similarity')
        else:
            print("There are currently no videos to recommend based on the user's current emotion level.")
            return

        # Print the user comment, emotion, and sum of negative emotion probabilities
        print(f"User comment: {user_comment['text']}")
        print(f"User emotion: {user_emotion}")
        print(f"User sum of negative emotion probabilities: {user_comment[negative_probs].sum()}")

    # Print the recommended video IDs and their emotion probabilities
    video_ids = recommended_videos['video_id'].tolist()
    print(f"Recommended video IDs: {video_ids}")
    if user_emotion in positive_emotions:
        print(f"Recommended video {user_emotion} probabilities: {recommended_videos['probability_' + user_emotion.lower()].tolist()}")
    else:
        print(f"Recommended video sum of negative emotion probabilities: {recommended_videos[negative_probs].sum(axis=1).tolist()}")
        print(f"Recommended video cosine similarity scores: {recommended_videos['cosine_similarity'].tolist()}")

    # Convert the video IDs to YouTube URLs
    youtube_urls = ['https://www.youtube.com/watch?v=' + video_id for video_id in video_ids]
    print(f"Recommended YouTube URLs: {youtube_urls}")

    return recommended_videos

In [172]:
# Create a function that recommends music videos based on user comment and emotion using the specified index
def recommend_music_videos_by_index(index, youtube_mv_transcripts = youtube_mv_transcripts, youtube_comments = youtube_comments):
    # Get the user comment and emotion based on the index
    user_comment = youtube_comments.iloc[index]
    user_emotion = user_comment['emotion']

    # Call the original function with the fetched comment and emotion
    recommend_music_videos(user_comment, user_emotion, youtube_mv_transcripts, youtube_comments)

In [173]:
# call the function with an index
recommend_music_videos_by_index(27386)

User comment: i lost my great grandpa even though i could not see him that much he was amazing and this song just makes me feel everything that i felt when i lost him
User emotion: Sadness
User sum of negative emotion probabilities: 0.9998680949211121
Recommended video IDs: ['qod03PVTLqk', 'AoAm4om0wTs', 'm4_9TFeMfJE', 'U6n2NcJ7rLc', 'je0roKRn3nY']
Recommended video sum of negative emotion probabilities: [0.7952914237976074, 0.6725053787231445, 0.6448014974594116, 0.7460570335388184, 0.6020493507385254]
Recommended video cosine similarity scores: [0.03645788368329114, 0.06054643781415544, 0.08317993742267622, 0.09016767111904954, 0.1102377029922241]
Recommended YouTube URLs: ['https://www.youtube.com/watch?v=qod03PVTLqk', 'https://www.youtube.com/watch?v=AoAm4om0wTs', 'https://www.youtube.com/watch?v=m4_9TFeMfJE', 'https://www.youtube.com/watch?v=U6n2NcJ7rLc', 'https://www.youtube.com/watch?v=je0roKRn3nY']


In [174]:
# Create a function that retrieves the transcript for a given video ID
def get_transcript(video_id, youtube_mv_transcripts = youtube_mv_transcripts):
    transcript = youtube_mv_transcripts[youtube_mv_transcripts['video_id'] == video_id]['transcript'].values[0]
    return transcript

In [175]:
# Call function to get transcript of music video
get_transcript('P3cffdsEXXw')

'golden  golden  golden  as i open my eyes  hold it  focus  hoping  take me back to the light  i know you were way too bright for me  i am hopeless  broken  so you wait for me in the sky  browns my skin just right  you are so golden  you are so golden  i am out of my head  and i know that you are scared  because hearts get broken  i do not want to be alone  i do not want to be alone  when it ends  do not want to let you know  i do not want to be alone  but i  i can feel it take a hold  i can feel you take control  of who i am and all i have ever known  loving you the antidote  golden  you are so golden  i do not want to be alone  you are so golden  you are so golden  i am out of my head  and i know that you are scared  because hearts get broken  i know that you are scared  because i am so open  you are so golden  i do not want to be alone  you are so golden  you are so golden  you are so golden  i am out of my head  and i know that you are scared  because hearts get broken '


---

<b> 5. Evaluation of Recommender System. </b>

The Emotion-centric recommender system I have build is functional but has room for improvement. The system’s ability to classify emotions in music videos and user comments is not always accurate. For instance, some music videos and comments that seem negative are classified as positive and vice versa. Furthermore, the system sometimes recommends music videos that don’t seem to match the user’s emotional state. For example, it might recommend a sad song to a user expressing joy or a sadder song to a user who is already feeling sad.

## Potential Weaknesses

1. **Limited Training Data:** The classification model was trained on a labeled dataset of tweets from Kaggle. This might not be representative of the language used in YouTube comments or music video lyrics.
2. **Complexity of Emotions:** Emotions can overlap and it can be challenging to discern emotions from text. A comment or a song could contain elements of multiple emotions, making it difficult for the model to classify accurately.
3. **Limited Scope of Analysis:** The system only analyzes the transcripts of the music videos, which might not fully capture the emotional content of the songs. The actual audio signals and the visuals of the music videos are not considered.
4. **Word Embeddings Limitations:** The GloVe word embeddings used in the model capture semantic relationships between words but are static and do not consider the context in which a word appears. This means the same word will have the same representation regardless of its context, which could limit the model’s ability to fully capture context-driven semantic word vectorization.
5. **Model Complexity:** The DistilBERT model, a transformer-based model, was expected to perform well due to its ability to understand the semantic meaning of words based on their context. However, it performed poorly, possibly due to its complexity and insufficient training.

## Future Work

- **Expand Training Data:** Consider increasing the data pool to include labeled comments from other social media platforms. This could help the model better understand the language used in YouTube comments and music video lyrics.
- **Explore Unsupervised Learning:** Explore using unsupervised learning to cluster the emotions of the text. This could potentially improve the accuracy of emotion classification.
- **Incorporate Audio and Visual Analysis:** Consider incorporating analysis of the audio signals and visuals of the music videos. This could provide a more comprehensive understanding of the emotional content of the songs.
- **Improve Word Embeddings:** Consider using context-aware word embeddings that can capture the semantic meaning of words based on their context.
- **Tune DistilBERT Model:** Future work could explore tuning the DistilBERT model or providing it with more training data to improve its performance.